In [45]:
 #Install Packages
 !pip install --quiet rioxarray
 !pip install --quiet rasterio

In [26]:
#Import Packages
import requests
import geopandas as gpd
import rasterio
import rasterio.plot
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import rioxarray as rxr
from rioxarray.merge import merge_arrays
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [3]:
#Set up for API Requests
# Set root URL for API requests
root_url = 'https://api.climateengine.org/'
# Authentication info for the API (INSERT YOUR OWN KEY)
headers = {'Authorization': ''}
# Google Storage bucket for storing output files (INSERT YOUR OWN BUCKET)
bucket_name = ''

In [7]:
#Bring in spatial file of interest (GeoJSON, SHP)
df = gpd.read_file('/content/Continental_US.shp')

 #Generate a bounding box around AOI
bbox = df.bounds

#Get values from box
sw_long = round(bbox.iat[0,0], 6)
sw_lat = round(bbox.iat[0,1], 6)
ne_long = round(bbox.iat[0,2], 6)
ne_lat = round(bbox.iat[0,3], 6)

#Generate Bounding Box Coordinates String
allot_bbox = [sw_long,sw_lat,ne_long,ne_lat]

print(allot_bbox)

[-124.725839, 24.498131, -66.949895, 49.384358]


In [34]:
#Set parameters + make request

#Select endpoint that exports a map of values to a Google Cloud storage bucket
endpoint = 'raster/export/counts'

# No need to include an extension on the export_filename below, .tif will be appended automatically.
filename = 'counts_temp'

#Set up parameters dictionary for API call
params = {
    'dataset': 'GRIDMET',
    'variable': 'tmmx',
    'operator': 'threshold_gte',
    'output': 'counts',
    'value_1': '35',
    'start_date': '2023-01-01',
    'end_date': '2023-12-31',
    'bounding_box': f'{allot_bbox}',
    'export_path': f'{bucket_name}/{filename}'
}

# Send API request
r = requests.get(root_url + endpoint, params=params, headers=headers, verify=False)
response = r.json()
print(response)



/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.climateengine.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'task_response': {'state': 'READY', 'description': 'counts_temp', 'creation_timestamp_ms': 1715110577303, 'update_timestamp_ms': 1715110577303, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'JFEG3ELK37PRTHHTS7LHW3UG', 'name': 'projects/climate-engine-pro/operations/JFEG3ELK37PRTHHTS7LHW3UG'}}


In [35]:

#Connect to GCS
#Get access to Google Cloud Storage Bucket
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
#Add Your Own Project ID
project_id = ''
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
#Bring in Raster from GCS
# Download the file from a given Google Cloud Storage bucket.
!gsutil cp gs://{bucket_name}/{filename}.tif /content/{filename}.tif

In [37]:
#Define file path and read in raster
file_path1 = f'/content/{filename}.tif'
rds = rxr.open_rasterio(file_path1)


In [ ]:
#Make Maps
# Generate Map
fig, ax = plt.subplots()
fig.set_size_inches(15, 13)
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
im = ax.imshow(rds[0], cmap ='Spectral_r', vmin = 0, vmax = 150)

#Add Color Bar
fig.colorbar(im, cax = cax, orientation = 'vertical')

#Add Title
ax.set_title('2023 GridMET Max Temp Counts: Days Greater Than or Equal To 95 Deg F ', fontsize =16)

# Remove Axes
ax.axis('off')

#Add Figure Export Option
plt.savefig('Counts.png', bbox_inches='tight')


#Show Map
plt.show()